In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
model_name = "play_model_1-best-3-pmx"
training_files = 3
data_files = glob.glob('play_data/play_model_1-20240906*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))
train_data['play_pos'] = train_data['play_pos'].astype('category')

In [3]:
train_data.head()

,run_id,key,play_pos,pos,partner_winning,lead_trumped,tricks_won,trumps_seen,turn_seen,next_seen,...,throw_off_void,throw_off_long,throw_off_low,throw_off_next,tricks_min,tricks_max,tricks_avg,points_min,points_max,points_avg
0,77127b,9♦ J♦ 10♣ Q♣ Q♠,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
1,77127b,9♦ J♦ 10♣ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
2,77127b,10♣ 9♦ J♦ Q♠ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0
3,77127b,10♣ 9♦ J♦ Q♣ Q♠,20,0,0,0,3,7,7,3,...,0,0,0,0,3,3,3.0,2,2,2.0
4,77127b,9♦ J♦ Q♠ 10♣ Q♣,23,0,0,0,2,7,7,4,...,0,0,0,0,2,2,2.0,-1,-1,-1.0


In [4]:
label = 'points_max'
problem_type = 'regression'
presets = 'best_quality'
train_data[label].describe()

count    575696.000000
mean         -0.089009
std           1.613302
min          -4.000000
25%          -1.000000
50%          -1.000000
75%           1.000000
max           4.000000
Name: points_max, dtype: float64

In [5]:
exclude = ['run_id', 'key']
outputs = ['tricks_min', 'tricks_max', 'tricks_avg', 'points_min', 'points_max', 'points_avg']
assert label in outputs
outputs.remove(label)

predictor = TabularPredictor(problem_type=problem_type, label=label, path=f"models/{model_name}")
predictor.fit(train_data.drop(columns=(exclude + outputs)), presets=presets)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       10.66 GB / 15.34 GB (69.5%)
Disk Space Avail:   114.57 GB / 452.95 GB (25.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to

(_ray_fit pid=69920) [1000]	valid_set's rmse: 0.690607
(_ray_fit pid=69920) [2000]	valid_set's rmse: 0.647808 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


(_dystack pid=69012) 	-0.6476	 = Validation score   (-root_mean_squared_error)
(_dystack pid=69012) 	178.93s	 = Training   runtime
(_dystack pid=69012) 	682.51s	 = Validation runtime
(_dystack pid=69012) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 249.9s of remaining time.
(_dystack pid=69012) 	Ensemble Weights: {'KNeighborsUnif_BAG_L1': 0.538, 'LightGBMXT_BAG_L1': 0.462}
(_dystack pid=69012) 	-0.5585	 = Validation score   (-root_mean_squared_error)
(_dystack pid=69012) 	0.53s	 = Training   runtime
(_dystack pid=69012) 	0.01s	 = Validation runtime
(_dystack pid=69012) Fitting 106 L2 models ...
(_dystack pid=69012) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 249.35s of the 249.31s of remaining time.
(_dystack pid=69012) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=7.08%)


(_ray_fit pid=70635) [1000]	valid_set's rmse: 0.463828 [repeated 6x across cluster]
(_ray_fit pid=70635) [2000]	valid_set's rmse: 0.452815 [repeated 8x across cluster]


(_dystack pid=69012) 	-0.4529	 = Validation score   (-root_mean_squared_error)
(_dystack pid=69012) 	201.51s	 = Training   runtime
(_dystack pid=69012) 	853.5s	 = Validation runtime
(_dystack pid=69012) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -63.85s of remaining time.
(_dystack pid=69012) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 1.0}
(_dystack pid=69012) 	-0.4529	 = Validation score   (-root_mean_squared_error)
(_dystack pid=69012) 	0.47s	 = Training   runtime
(_dystack pid=69012) 	0.0s	 = Validation runtime
(_dystack pid=69012) AutoGluon training complete, total runtime = 960.19s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 40.4 rows/s (63967 batch size)
(_dystack pid=69012) TabularPredictor saved. To load, use: predictor = TabularPredictor.load("models/play_model_1-best-3-pmx/ds_sub_fit/sub_fit_ho")
(_dystack pid=69012) Deleting DyStack predictor artifacts (clean_up_fits=True) ...
Leaderboard on holdout data (DyStack):
 

In [6]:
test_data = TabularDataset(data_files[5])[:100000]
y_pred = predictor.predict(test_data[:1000])
y_pred.head(10)

Loaded data from: play_data/play_model_1-20240906_173932.tsv | Columns = 55 / 55 | Rows = 194348 -> 194348


0   -1.466322
1   -1.481569
2   -1.538257
3   -1.466322
4   -1.538257
5   -1.481569
6   -1.466322
7   -1.481569
8   -1.466322
9   -1.538257
Name: points_max, dtype: float32

In [7]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -0.9539550853192876,
 'mean_squared_error': -0.9100303048065294,
 'mean_absolute_error': -0.5504248784830748,
 'r2': 0.6376667491949933,
 'pearsonr': 0.8002402134324312,
 'median_absolute_error': -0.12177431583404541}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,-0.773636,-0.573006,root_mean_squared_error,163.759755,2450.189168,431.396199,163.759755,2450.189168,431.396199,1,True,3
1,LightGBMXT_BAG_L2,-0.953955,-0.353992,root_mean_squared_error,563.922112,5956.163934,944.240227,199.879193,2353.367555,512.085067,2,True,5
2,WeightedEnsemble_L3,-0.953955,-0.353992,root_mean_squared_error,563.926262,5956.169453,944.682121,0.004149,0.005520,0.441894,3,True,6
3,WeightedEnsemble_L2,-0.965137,-0.495846,root_mean_squared_error,264.561413,3020.545247,432.331087,0.002576,0.005483,0.558257,2,True,4
4,KNeighborsUnif_BAG_L1,-1.413234,-0.648315,root_mean_squared_error,99.484082,582.256614,0.382330,99.484082,582.256614,0.382330,1,True,1
5,KNeighborsDist_BAG_L1,-1.416334,-0.588007,root_mean_squared_error,100.799082,570.350596,0.376631,100.799082,570.350596,0.376631,1,True,2
